# CAN-DB 관리

In [ ]:
# HOW TO USE
# 자체제어기 Excel 테이블을 클립보드로 복사(Ctrl+C) 한 뒤 아래 코드를 수행
# 기본 저장 경로: SVN 내 CAN 관리 폴더
# @saveAs: 저장할 경로 선택
from pyems.candb.vcs import CanDbVersionControl
vcs = CanDbVersionControl()
vcs.clipbd2db()

# 자동화 모델 생성: %ComDef *_HEV

In [6]:
# HOW TO USE
# @engine_spec 결정: "ICE" / "HEV"
# (필요 시) 제외 메시지에 대한 Customize
from pyems.candb.reader import CanDb
from cannect.can.ascet.ComDef import ComDef

db = CanDb()
engine_spec = "ICE"

# DB CUSTOMIZE ------------------------------------------------------
exclude_ecus = ["EMS", "CVVD", "MHSG", "NOx"]
if engine_spec == "ICE":
    exclude_ecus += ["BMS", "LDC"]
db = db[~db["ECU"].isin(exclude_ecus)]

# db = db[db["Status"] != "TSW"] # TSW 제외
# db = db[~db["Requirement ID"].isin(["VCDM CR10777888"])] # 특정 CR 제외
# db = db[~db["Required Date"].isin(["2024-08-27"])] # 특정 일자 제외
db = db[~db["Message"].isin([ # 특정 메시지 제외
    "ADAS_UX_02_50ms",
    "HU_CLU_USM_01_00ms",
    "HU_CLU_USM_E_01",
    "HU_NAVI_05_200ms",
    "HU_NAVI_V2_3_POS_PE",
    "HU_CLOCK_01_1000ms",
    "HU_CLOCK_PE_02",
])]
# db.revision = "TEST SW" # 공식SW는 주석 처리
# DB CUSTOMIZE END --------------------------------------------------
db = db.to_developer_mode(engine_spec)

model = ComDef(
    db=db,
    engine_spec=engine_spec,
    base_model="", # 미 입력 시, SVN 최신 모델 사용
    exclude_tsw=True,
)
model.autorun()

PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\Users\\Administrator\\Downloads\\ComDef\\log.txt'

# 자동화 모델 생성: %ComRx *_HEV

# 자동화 모델 생성: 진단 모델

In [7]:
import sys
sys.path.append(r"D:\CANNECT")  # add project root

from pyems.environ import ENV
from pyems.ascet import ProjectIO
from pyems.candb import CanDb
from cannect.can.preset import DIAGNOSIS_HEV
from cannect.can.ascet.CanDiag import Template

proj = ProjectIO(ENV['MODEL']['HNB_GASOLINE'])
comm = proj.bcTree(29)
CANDB = CanDb()

# @unit [str]
# : 모델명 입력 시, 단일 모델 생성
# : 모델명 공백 시, 전체 모델 생성
# * 수기로 수정해야하는 사항을 꼭 파악한 후 반영하세요.
unit = ""
# unit = ''
for model, messages in DIAGNOSIS_HEV.items():
    if unit and unit != model:
        continue
    tree = comm[comm['file'] == f'{model}.zip']
    if tree.empty:
        raise KeyError(f'MODEL: {model}.zip NOT FOUND')
    if len(tree) >= 2:
        print(tree)
        n = input(f'DUPLICATED MODELS FOUND, SELECT INDEX OF THE LIST: ')
        tree = tree.loc[int(n)]
    else:
        tree = tree.iloc[0]

    template = Template(CANDB, tree['path'], *messages)
    template.create()

2025-12-17 10:57:27 %{CanFDACUD_HEV} MODEL GENERATION
2025-12-17 10:57:27 >>> DB VERSION: r21653@01
2025-12-17 10:57:27 >>> BASE MODEL: E:\SVN\model\ascet\trunk\HNB_GASOLINE\_29_CommunicationVehicle\CANInterface\ACU\MessageDiag\CanFDACUD_HEV\CanFDACUD_HEV.zip
2025-12-17 10:57:27 >>> COPY BASE MODEL TO TEMPLATE
2025-12-17 10:57:27 >>> GENERATE HIERARCHY BY MESSAGES N=1
2025-12-17 10:57:27 >>> ... [1 / 1] IMU_01_10ms: 
2025-12-17 10:57:27 >>> COPY DSM LIBRARY IMPLEMENTATION
2025-12-17 10:57:27 >>> COPY CALIBRATION DATA FROM BASE MODEL
2025-12-17 10:57:27 >>> RUN EXCEPTION HANDLING
2025-12-17 10:57:27 >>> ... NO EXCEPTION FOUND
2025-12-17 10:57:27 %{CanFDBDCD_HEV} MODEL GENERATION
2025-12-17 10:57:27 >>> DB VERSION: r21653@01
2025-12-17 10:57:27 >>> BASE MODEL: E:\SVN\model\ascet\trunk\HNB_GASOLINE\_29_CommunicationVehicle\CANInterface\BDC\MessageDiag\CanFDBDCD_HEV\CanFDBDCD_HEV.zip
2025-12-17 10:57:27 >>> COPY BASE MODEL TO TEMPLATE
2025-12-17 10:57:27 >>> GENERATE HIERARCHY BY MESSAGES 

KeyError: 'HCU_11_H_00ms NOT EXIST IN CAN DB.'

# TEST CASE

## TEST REPORT 생성(Excel)

## XDA 생성

### [XDA 생성] 진단

In [ ]:
from cannect.can.preset import DIAGNOSIS_ICE
from cannect.can.testcase.xda.diagnosis import XdaDiagnosis
import os

root = r'\\kefico\keti\ENT\Softroom\Temp\J.H.Lee\00 CR\CR10785931 J1979-2 CAN 진단 대응 ICE CANFD\08_Verification'
for model, messages in DIAGNOSIS_ICE.items():
    dat = os.path.join(root, rf'Data/{model}_Det_Diag.dat')
    for message in messages:
        try:
            print(model, message, end=": ")
            xda = XdaDiagnosis(dat=dat, message=message)
            xda.create(os.path.join(root, 'xda'))
            print("SUCCESS")
        except Exception as e:
            print("FAILED", e)